# AirBnb NY Listing Price Prediction: Data Preparation

In [0]:
import pandas as pd
import os
from pyspark.sql.functions import *

### Catalog, Schema Creation

In [0]:
#catalog_ = f"price_prediction"
#schema_ = f"ny_listing"
catalog_ = os.getenv('CATALOG_NAME')
schema_ = os.getenv('SCHEMA_NAME')

In [0]:
#spark.sql("CREATE CATALOG IF NOT EXISTS "+catalog_)
#spark.sql("CREATE SCHEMA IF NOT EXISTS "+catalog_+"."+schema_)

In [0]:
spark.sql("USE CATALOG "+catalog_)
spark.sql("USE SCHEMA "+schema_)

### Data Ingestion from Volume [Bronze]

In [0]:
path_ = f"dbfs:/Volumes/"+catalog_+"/"+schema_+"/ny_listing/NY_Listing_cleaned.csv"
bronze_ = (spark.
           read.
           option("header", True).
           #option("delimiter", ",").
           option("encoding", "utf-8").
           csv(path_).
           createOrReplaceTempView("bronze_v"))

display(spark.sql("SELECT * FROM bronze_v LIMIT 10;"))


In [0]:
%sql
DROP TABLE IF EXISTS bronze_data;
CREATE TABLE bronze_data AS
  SELECT id, NAME as name, `host id` as host_id, host_identity_verified, `host name` as host_name, `neighbourhood group` as neighbourhood_group
    ,neighbourhood, lat, long, country,`country code` as country_code, instant_bookable, cancellation_policy, `room type` as room_type,`Construction year` as construction_year, price, `service fee` as service_fee, `minimum nights` as minimum_nights, `number of reviews` as number_of_reviews, `last review` as last_review, `reviews per month` as reviews_per_month, `review rate number` as review_rate_number,`calculated host listings count` as calculated_host_listings_count, `availability 365` as availability_365, house_rules, license
  FROM bronze_v;

### Data Cleaning [Silver]

Cleaning steps required:
* Remove duplicates
* Remove invalid characters from price column (e.g., $ ,)
* Remove invalid availabilities (e.g., 0 or >365)

#### 1,2-Remove Duplicates & Convert Column Formats

In [0]:
%sql
-- Duplicates
with d0 (
select distinct * from bronze_data)
select "Tot Records" as count, count(*) value_ from bronze_data
  union select "Unique Records" as count, count(*) value_ from d0;

In [0]:
%sql
DROP TABLE IF EXISTS silver_data;
CREATE TABLE silver_data AS
with d0 as (
  -- Remove duplicates and initial cleaning
  select distinct *
  from bronze_data
  where (host_identity_verified in ('verified', 'unconfirmed') or host_identity_verified is null) 
  and availability_365 is not null
  and id not in (' Laundry in building  8 mints from A C trains')
  and construction_year is not null
  and minimum_nights is not null and minimum_nights not like '$%'
  and number_of_reviews is not null and number_of_reviews not like '$%'
  and review_rate_number is not null and review_rate_number not like '$%' and review_rate_number not like '%/%'
  and calculated_host_listings_count is not null and calculated_host_listings_count not like '%.%'
  -- Removing a few problematic data
), d1 as (
  -- Clean up some fields
  select id,	`name`,	host_id,	host_identity_verified,	host_name
  ,case when neighbourhood_group = 'brookln' then 'Brooklyn' when neighbourhood_group = 'manhatan' then 'Manhattan' else neighbourhood_group end as neighbourhood_group
  ,	neighbourhood,	lat,	long,	country,	country_code, instant_bookable, cancellation_policy, room_type, construction_year, price, service_fee
  ,cast(case when abs(minimum_nights)>365 then 365 else abs(minimum_nights) end as int) as minimum_nights
  , number_of_reviews, last_review,	reviews_per_month,	review_rate_number, calculated_host_listings_count
  ,cast(case when abs(availability_365)>365 then 365 else abs(availability_365) end as int) as availability_365
  , house_rules, license
  from d0
)
-- Clean and convert numerical fields
  select
    cast(id as int) id
    ,`name`
    ,cast(trim(host_id) as int) host_id
    ,host_identity_verified, host_name, neighbourhood_group, neighbourhood
    ,cast(lat as float) lat
    ,cast(long as float) long
    ,country,	country_code, instant_bookable, cancellation_policy, room_type
    ,cast(construction_year as int) construction_year
    ,cast(minimum_nights as int) minimum_nights
    ,cast(number_of_reviews as int) number_of_reviews
    ,to_timestamp(trim(last_review), 'M/d/yyyy') last_review
    ,cast(reviews_per_month as float) reviews_per_month
    ,cast(review_rate_number as int) review_rate_number
    ,cast(calculated_host_listings_count as int) calculated_host_listings_count
    ,cast(availability_365 as int) availability_365
    ,house_rules, license
    ,cast(
      case when price like '%$%' then replace(price, '$', '')
            when price like '%,%' then replace(price, ',', '')
            else price end
      as float) price
    ,cast(
      case when service_fee like '%$%' then replace(service_fee, '$', '')
            when service_fee like '%,%' then replace(service_fee, ',', '')
            else service_fee end
      as float) service_fee      
from d1;

#### (2-Convert Column Formats)

In [0]:
#silver_step1 = spark.sql("SELECT * from silver_data_1").toPandas()

In [0]:
# Int columns
'''
silver_step1['id'] = silver_step1['id'].astype(int)
silver_step1['host_id'] = silver_step1['host_id'].astype(int)
silver_step1['construction_year'] = silver_step1['construction_year'].astype(int)
silver_step1['minimum_nights'] = silver_step1['minimum_nights'].astype(int)
silver_step1['number_of_reviews'] = silver_step1['number_of_reviews'].astype(int)
silver_step1['review_rate_number'] = silver_step1['review_rate_number'].astype(int)
silver_step1['calculated_host_listings_count'] = silver_step1['calculated_host_listings_count'].astype(int)
silver_step1['availability_365'] = silver_step1['availability_365'].astype(int)

#float
silver_step1['lat'] = silver_step1['lat'].astype(float)
silver_step1['long'] = silver_step1['long'].astype(float)
silver_step1['reviews_per_month'] = silver_step1['reviews_per_month'].astype(float)

#date
silver_step1['last_review'] = pd.to_datetime(silver_step1['last_review'])

# Cleaning and float
silver_step1['service_fee'] = silver_step1['service_fee'].str.replace('$', '')
silver_step1['service_fee'] = silver_step1['service_fee'].str.replace(',', '')
silver_step1['service_fee'] = silver_step1['service_fee'].astype(float)

silver_step1['price'] = silver_step1['price'].str.replace('$', '')
silver_step1['price'] = silver_step1['price'].str.replace(',', '')
silver_step1['price'] = silver_step1['price'].astype(float)
'''

In [0]:
# Save into silver table
#silver_step1 = spark.createDataFrame(silver_step1).createOrReplaceTempView("silver_v")

In [0]:
'''
%sql
DROP TABLE IF EXISTS silver_data;
CREATE TABLE silver_data AS
  select * from silver_v;
'''

### [Gold data] Finalize data preparation for ML

Steps required:

* Remove uninformative columns (same value on all rows or different values on every row, columns with >50% Null values)
* Remove NAs
* Encode required categorical columns


#### 1-Remove Uninformative Columns & NAs

In [0]:
gold1_sdf = spark.sql("SELECT * from silver_data")
gold1 = gold1_sdf.toPandas()
display(gold1.info())

In [0]:
gold1.describe(percentiles=[.25, .5, .75])

In [0]:
# Drop columns
to_drop = ['name','host_id','house_rules','license','host_name', ## Too many unique values
           'last_review', 'reviews_per_month',  ## Too many null values
           'country','country_code', ## Too few unique values
           'service_fee' ## Unlikely this info is available when we predict the price
           ]
for c in to_drop:
  gold1_sdf = gold1_sdf.drop(c)

gold_step1 = gold1_sdf.toPandas()
display(gold_step1.info())

In [0]:
# Drop NAs
for c in list(gold1_sdf.columns):
  len_pre = gold1_sdf.count()
  gold1_sdf = gold1_sdf.dropna(subset=c)
  len_post = gold1_sdf.count()
  if len_pre>len_post:
    print(":: On col %s, removed %4d NAs" % (c, len_pre - len_post))

#### 2-Encode Categorical Columns

Since we will be using tree based algorithms, we will do an **Ordinal Encoding** and also a **One Hot Encoding**.

In [0]:
gold1_sdf_bkp = gold1_sdf.select("*")
display(gold1_sdf_bkp.head(1))

In [0]:
gold_step2 = gold1_sdf.toPandas()
col_to_exclude = ['id']
col_ordinal_encoding = {}

## Check values per column
for col_ in gold_step2.columns:
  if gold_step2[col_].dtype == 'object' and col_ not in col_to_exclude:
    gold_groupby = gold_step2.groupby(col_).agg(records=('price','count'), avg_price=('price', 'mean'), std_price=('price', 'std'))
    gold_groupby.reset_index(inplace=True)
    print(":: %s has %2d unique values" % (col_, len(gold_groupby)))
    gold_groupby.sort_values(by=col_, inplace=True)
    display(gold_groupby)
    ## Encode unique values
    values_ = list(gold_groupby[col_])
    col_ordinal_encoding[col_] = {categ_value: str(id+1) for id, categ_value in enumerate(values_)} # Make str to avoid mixed type replacement errors
    print("\n")

##### 2.1 Ordinal Encoding

In [0]:
col_ordinal_encoding

In [0]:
## Create encoded columns on the dataframe
for k, encod_dict in col_ordinal_encoding.items():
  gold1_sdf = (
    gold1_sdf
    .withColumn(f"{k}_encoded", col(k))
    .replace(to_replace=encod_dict, subset=[k+"_encoded"]) # Map dictionary to a column
    .withColumn(k+"_encoded", col(k+"_encoded").cast('float'))
  )

In [0]:
## Persist encoded column info
gold1_sdf.createOrReplaceTempView("gold_v")

In [0]:
%sql
DROP TABLE IF EXISTS gold_data;
CREATE TABLE gold_data AS
Select 
  id
  ,cast(host_identity_verified_encoded as float) as host_identity_verified_encoded
  ,cast(neighbourhood_group_encoded as float) as neighbourhood_group_encoded
  ,cast(neighbourhood_encoded as float) as neighbourhood_encoded
  ,cast(lat as float)
  ,cast(long as float)
  ,cast(instant_bookable_encoded as float) as instant_bookable_encoded
  ,cast(cancellation_policy_encoded as float) as cancellation_policy_encoded
  ,cast(room_type_encoded as float) as room_type_encoded
  ,cast(construction_year as float) construction_year
  ,cast(minimum_nights as float) minimum_nights
  ,cast(number_of_reviews as float) number_of_reviews
  ,cast(review_rate_number as float) review_rate_number
  ,cast(calculated_host_listings_count as float) calculated_host_listings_count
  ,cast(availability_365 as float) availability_365
  ,cast(price as float) price
from gold_v;

In [0]:
%sql
SELECT * from gold_data limit 10;

##### 2.2-OneHotEncoding

In [0]:
gold1_sdf_bkp.createOrReplaceTempView("gold_v")

In [0]:
%sql
-- Approach used for the one hot encoding
DROP TABLE IF EXISTS gold_data_ohe;
CREATE TABLE gold_data_ohe AS
Select 
  id
  ,cast(case when host_identity_verified = 'verified' then 1 else 0 end as float) as is_host_identity_verified
  /* Neighborhood groups = Bronx, Brooklyn, Manhattan, Queens, Staten Island */
  ,cast(case when neighbourhood_group = 'Bronx' then 1 else 0 end as float) is_neighbourhood_group_Bronx
  ,cast(case when neighbourhood_group = 'Brooklyn' then 1 else 0 end as float) is_neighbourhood_group_Brooklyn
  ,cast(case when neighbourhood_group = 'Manhattan' then 1 else 0 end as float) is_neighbourhood_group_Manhattan
  ,cast(case when neighbourhood_group = 'Queens' then 1 else 0 end as float) is_neighbourhood_group_Queens
  /* dropping neighbourhood column */
  ,cast(lat as float)
  ,cast(long as float)
  ,cast(case when instant_bookable = 'TRUE' then 1 else 0 end as float) is_instant_bookable
  /* cancellation_policy = flexible, moderate, strict */
  ,cast(case when cancellation_policy = 'flexible' then 1 else 0 end as float) is_cancellation_policy_flexible
  ,cast(case when cancellation_policy = 'strict' then 1 else 0 end as float) is_cancellation_policy_strict
  /* room_type = Entire home/apt, Hotel room, Private room, Shared room */
  ,cast(case when room_type = 'Entire home/apt' then 1 else 0 end as float) is_room_type_Entire
  ,cast(case when room_type = 'Private room' then 1 else 0 end as float) is_room_type_Privateroom
  ,cast(case when room_type = 'Shared room' then 1 else 0 end as float) is_room_type_Sharedroom
  ,cast(construction_year as float) construction_year
  ,cast(minimum_nights as float) minimum_nights
  ,cast(number_of_reviews as float) number_of_reviews
  ,cast(review_rate_number as float) review_rate_number
  ,cast(calculated_host_listings_count as float) calculated_host_listings_count
  ,cast(availability_365 as float) availability_365
  ,cast(price as float) price
from gold_v;

### Final counts

In [0]:
display(spark.sql("select count(*) from bronze_data;"))
display(spark.sql("select count(*) from silver_data;"))
display(spark.sql("select count(*) from gold_data;"))
display(spark.sql("select count(*) from gold_data_ohe;"))